### Background
As smart phone penetration reaches the hundreds of millions mark, O2O (Online to Offline) requires businesses to have a strong presence both offline and online. APPs with O2O capabilities accumulate daily consumer behaviour and location data that require big data and commercial operations management. The competition at hand focuses on coupon redemption rates. Sending coupons is a general O2O marketing tool used to activate existing customers and attract new ones. While customers are happy to receive coupons that they want, they are frustrated when receiving coupons that they do not need. For merchants, sending unwanted coupons may erode brand equity and hinder marketing expense forecasting. Targeted marketing is an important technology to increase the coupon redemption rate, providing relevant discounts to customers and effective marketing tools to businesses. The competition provides participants with abundant O2O data in this field and expects contestants to predict whether the customer will use the coupon within a specified time frame.
### Data
This competition provides real online and offline user consumption data from January 1, 2016 to June 30, 2016. The contestants are expected to predict the probability of customers redeeming a coupon within 15 days of receiving it.
Note: To protect the privacy of users and merchants, data is desensitized and under biased sampling.
### Evaluation
The results are evaluated based on the average AUC value. That is, the AUC value is calculated for every coupon_id. The average of each AUC value is the evaluation score. More information on AUC value calculation method on wikipedia.


In [1]:
import time
now = time.time()

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

### Online and Offline Training data

In [3]:
df_on = pd.read_csv('DataSets/ccf_online_stage1_train.csv')
df_off = pd.read_csv('DataSets/ccf_offline_stage1_train.csv')

In [4]:
print("Online Training Data Sample\nShape:"+str(df_on.shape))
df_on.head()

Online Training Data Sample
Shape:(11429826, 7)


,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513.0,NaN
1,13740231,34805,1,NaN,NaN,NaN,20160321.0
2,14336199,18907,0,NaN,NaN,NaN,20160618.0
3,14336199,18907,0,NaN,NaN,NaN,20160618.0
4,14336199,18907,0,NaN,NaN,NaN,20160618.0


In [5]:
print("Offline Training Data Sample\nShape:"+str(df_off.shape))
df_off.head()

Offline Training Data Sample
Shape:(1754884, 7)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


### Test Data (Offline)

In [6]:
df_test = pd.read_csv('DataSets/ccf_offline_stage1_test_revised.csv')
print("Testing Data(Offline) Sample\nShape:"+str(df_test.shape))
df_test.head()

Testing Data(Offline) Sample
Shape:(113640, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


#### Converting Date to DateTime format

In [7]:
#Online Training Data
df_on['Date'] = pd.to_datetime(df_on["Date"],format='%Y%m%d')
df_on['Date_received'] = pd.to_datetime(df_on["Date_received"],format='%Y%m%d')

#Offline Training Data
df_off['Date'] = pd.to_datetime(df_off["Date"],format='%Y%m%d')
df_off['Date_received'] = pd.to_datetime(df_off["Date_received"],format='%Y%m%d')

#Testing Data
df_test['Date_received'] = pd.to_datetime(df_test["Date_received"],format='%Y%m%d')

### Removing Duplicates from Online and Offline Training Data

In [8]:
#Removing duplicates and giving frequency counts(Count) to each row

#Online
x = 'g8h.|$hTdo+jC9^@'    
df_on_unique = (df_on.fillna(x).groupby(['User_id', 'Merchant_id', 'Action', 'Coupon_id', 'Discount_rate',
       'Date_received', 'Date']).size().reset_index()
               .rename(columns={0 : 'Count'}).replace(x,np.NaN))
df_on_unique["Date_received"]=pd.to_datetime(df_on_unique["Date_received"])
df_on_unique["Date"]=pd.to_datetime(df_on_unique["Date"])

print("Online Training Data Shape:"+str(df_on_unique.shape))

Online Training Data Shape:(5822543, 8)


In [9]:
#Offline
x = 'g8h.|$hTdo+jC9^@'   #garbage value for nan values 
df_off_unique = (df_off.fillna(x).groupby(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date']).size().reset_index()
               .rename(columns={0 : 'Count'}).replace(x,np.NaN))
df_off_unique["Date_received"]=pd.to_datetime(df_off_unique["Date_received"])
df_off_unique["Date"]=pd.to_datetime(df_off_unique["Date"])

print("Offline Training Data Shape:"+str(df_off_unique.shape))

Offline Training Data Shape:(1716991, 8)


#### Filling Nan for Distance (OFFLINE)

In [10]:
df_off_unique['Distance'].fillna(df_off_unique['Distance'].mean(), inplace=True)
df_off_unique['Distance'] = df_off_unique.Distance.astype(int)

### Converting Discount Ratio to Rate

In [11]:
#Funtion to convert discount ratio to discount rate
def convert_discount(discount):
    values = []
    for i in discount:
        if ':' in i:
            i = i.split(':')
            rate = round((int(i[0]) - int(i[1]))/int(i[0]),3)
            values.append([int(i[0]),int(i[1]),rate])
        elif '.' in i:
            i = float(i)
            x = 100*i
            values.append([100,int(100-x),i])
            
    discounts = dict(zip(discount,values))      
    return discounts
    

# convert_discount(list(df_of['Discount_rate']))

In [12]:
#ONLINE DATA
df_on_coupon = df_on_unique[(df_on_unique['Coupon_id'].isna()==False) & (df_on_unique['Coupon_id']!='fixed')]
discounts_online = list(df_on_coupon['Discount_rate'].unique())
df_on_coupon.loc[:,('Discount')] = df_on_coupon.loc[:,('Discount_rate')] 
df_on_coupon.loc[:,('Discount_rate')] = df_on_coupon.loc[:,('Discount')].map(convert_discount(discounts_online))
df_on_coupon[['Original_price','Discounted_price','Rate']] = pd.DataFrame(df_on_coupon.Discount_rate.values.tolist(), index= df_on_coupon.index)
df_on_coupon.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,Count,Discount,Original_price,Discounted_price,Rate
0,4,25104,2,100145044,"[100, 10, 0.9]",2016-03-31,NaT,1,100:10,100,10,0.900
14,209,27710,1,100081876,"[150, 10, 0.933]",2016-05-14,2016-05-14,1,150:10,150,10,0.933
44,236,53305,2,100075632,"[200, 10, 0.95]",2016-02-21,NaT,1,200:10,200,10,0.950
45,236,53305,2,100181719,"[50, 1, 0.98]",2016-02-21,NaT,1,50:1,50,1,0.980
46,236,53305,2,100214552,"[150, 5, 0.967]",2016-02-21,NaT,1,150:5,150,5,0.967


In [13]:
#OFFLINE DATA
df_off_coupon = df_off_unique[(df_off_unique['Coupon_id'].isna()==False)].copy()
discounts_offline = list(df_off_coupon['Discount_rate'].unique())
df_off_coupon.loc[:,('Discount')] = df_off_coupon.loc[:,('Discount_rate')] 
df_off_coupon['Discount_rate'] = df_off_coupon['Discount'].map(convert_discount(discounts_offline))
df_off_coupon[['Original_price','Discounted_price','Rate']] = pd.DataFrame(df_off_coupon.Discount_rate.values.tolist(), index= df_off_coupon.index)
df_off_coupon.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Count,Discount,Original_price,Discounted_price,Rate
0,4,1433,8735.0,"[30, 5, 0.833]",10,2016-02-14,NaT,1,30:5,30,5,0.833
1,4,1469,2902.0,"[100, 5, 0.95]",10,2016-06-07,NaT,1,0.95,100,5,0.950
2,35,3381,1807.0,"[300, 30, 0.9]",0,2016-01-30,NaT,1,300:30,300,30,0.900
3,35,3381,9776.0,"[10, 5, 0.5]",0,2016-01-29,NaT,1,10:5,10,5,0.500
4,35,3381,11951.0,"[200, 20, 0.9]",0,2016-01-29,NaT,1,200:20,200,20,0.900


In [14]:
#TEST DATA
discounts_test = list(df_test['Discount_rate'].unique())
df_test.loc[:,('Discount')] = df_test.loc[:,('Discount_rate')] 
df_test['Discount_rate'] = df_test['Discount'].map(convert_discount(discounts_test))
df_test[['Original_price','Discounted_price','Rate']] = pd.DataFrame(df_test.Discount_rate.values.tolist(), index= df_test.index)
df_test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Discount,Original_price,Discounted_price,Rate
0,4129537,450,9983,"[30, 5, 0.833]",1.0,2016-07-12,30:5,30,5,0.833
1,6949378,1300,3429,"[30, 5, 0.833]",NaN,2016-07-06,30:5,30,5,0.833
2,2166529,7113,6928,"[200, 20, 0.9]",5.0,2016-07-27,200:20,200,20,0.900
3,2166529,7113,1808,"[100, 10, 0.9]",5.0,2016-07-27,100:10,100,10,0.900
4,6172162,7605,6500,"[30, 1, 0.967]",2.0,2016-07-08,30:1,30,1,0.967


# FEATURES

In [15]:
users_level = pd.read_csv('DataSets/DatasetsCreated/user_level.csv')
users_level.head()

,User_id,Tag,User_Released,User_Redeemed,User_Ratio,User_Buys,Purchaser,UserMerchantCount,DayList,User_Redeemed_Buy
0,4,0,3.0,0.0,0.0,1,0,4,[68],0.0
1,35,1,4.0,0.0,0.0,0,0,1,[],0.0
2,36,0,2.0,0.0,0.0,1,0,3,[20],0.0
3,64,0,1.0,0.0,0.0,3,0,3,"[147, 158]",0.0
4,110,1,3.0,0.0,0.0,0,0,3,[],0.0


In [16]:
merchants_level = pd.read_csv('DataSets/DatasetsCreated/merchant_level.csv')
merchants_level.head()

,Merchant_id,Merchant_Redeemed,Merchant_Ratio,Merchant_AvgDistance,Merchant_Popular,Merchant_AvgRate,AvgDailyUsers,VisitList,UniqueUsersCount,Merchant_Buys,MerchantRedemptionList,Merchant_Redeemed_Buy
0,1433,726.0,0.04,3.927992,1,0.810455,49.697802,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",19340,9045,"[24, 25, 27, 29, 30, 31, 33, 34, 35, 36, 37, 3...",0.080265
1,1469,675.0,0.05,2.617818,1,0.707819,74.527473,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",13702,13576,"[9, 10, 14, 15, 20, 21, 30, 31, 61, 64, 65, 66...",0.049720
2,3381,2473.0,0.02,2.690394,1,0.866463,119.587912,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",108018,21829,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",0.113290
3,1041,402.0,0.05,2.846591,1,0.831778,19.510989,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",7966,3553,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, ...",0.113144
4,5717,293.0,0.02,2.255078,1,0.751612,21.571429,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",12506,3927,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",0.074612


In [17]:
coupons_level = pd.read_csv('DataSets/DatasetsCreated/coupon_level.csv')
coupons_level.head()

,Coupon_id,Coupon_Released,Coupon_Redeemed,Coupon_Ratio,Duration,RedemptionList
0,1,5,1,0.20,24,[154]
1,10,32,15,0.47,18,"[136, 138, 139, 140, 141, 142, 143, 145, 146, ..."
2,100,7,1,0.14,22,[142]
3,1000,38,4,0.11,29,"[31, 32, 35, 37]"
4,10000,17,11,0.65,47,"[8, 12, 15, 16, 17, 25, 26, 27, 28, 31, 49]"


In [18]:
rates_level = pd.read_csv('DataSets/DatasetsCreated/rate_level.csv')
# rates_level['Rate'] = rates_level['Rate'].apply(lambda x: round(x,2))
# df_off_coupon['Rate'] = df_off_coupon['Rate'].apply(lambda x: round(x,2))
rates_level.head()

,Rate,Rate_Releases,Rate_Redeemed,Rate_Ratio
0,0.200,81,6,0.074074
1,0.333,45003,3920,0.087105
2,0.375,4,4,1.000000
3,0.400,11395,2529,0.221939
4,0.500,142664,18480,0.129535


In [19]:
date_level = pd.read_csv('DataSets/DatasetsCreated/date_level.csv')
date_level['Date_received'] =  date_level['Date_received'].astype('datetime64[ns]') 
date_level.head()

,Date_received,ReleasesCount,ImpDay,Weekend,DayOfWeek
0,2016-01-01,553,0,0,4
1,2016-01-02,541,0,1,5
2,2016-01-03,529,0,1,6
3,2016-01-04,574,0,0,0
4,2016-01-05,677,0,0,1


In [20]:
date_level_test = pd.read_csv('DataSets/DatasetsCreated/date_level_test.csv')
date_level_test['Date_received'] =  date_level_test['Date_received'].astype('datetime64[ns]') 
date_level_test.head()

,Date_received,ReleasesCount,ImpDay,Weekend,DayOfWeek
0,2016-07-01,3808,0,0,4
1,2016-07-02,3831,0,1,5
2,2016-07-03,5491,0,1,6
3,2016-07-04,5121,0,0,0
4,2016-07-05,4693,0,0,1


In [21]:
df_off_coupon.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Count,Discount,Original_price,Discounted_price,Rate
0,4,1433,8735.0,"[30, 5, 0.833]",10,2016-02-14,NaT,1,30:5,30,5,0.833
1,4,1469,2902.0,"[100, 5, 0.95]",10,2016-06-07,NaT,1,0.95,100,5,0.950
2,35,3381,1807.0,"[300, 30, 0.9]",0,2016-01-30,NaT,1,300:30,300,30,0.900
3,35,3381,9776.0,"[10, 5, 0.5]",0,2016-01-29,NaT,1,10:5,10,5,0.500
4,35,3381,11951.0,"[200, 20, 0.9]",0,2016-01-29,NaT,1,200:20,200,20,0.900


### User, Discounted and Original Price Redemption Rate

In [22]:
user_discount_redemption=pd.DataFrame(df_off_coupon.groupby(['User_id','Discounted_price','Original_price'])
                                      ['Date_received','Date'].count()).reset_index()
user_discount_redemption.columns= ['User_id','Discounted_price','Original_price',
                                   'Discounted_Released','Discounted_Redeemed']
user_discount_redemption['User_discount_Ratio'] = user_discount_redemption['Discounted_Redeemed']/user_discount_redemption['Discounted_Released']
user_discount_ratio = user_discount_redemption[['User_id','Discounted_price','Original_price',
                                                 'Discounted_Redeemed','User_discount_Ratio']]
user_discount_ratio.head()


,User_id,Discounted_price,Original_price,Discounted_Redeemed,User_discount_Ratio
0,4,5,30,0,0.0
1,4,5,100,0,0.0
2,35,5,10,0,0.0
3,35,20,200,0,0.0
4,35,30,300,0,0.0


### User-Coupon Coupon Redemption Rate

In [23]:
coupon_user_redemption = pd.DataFrame(df_off_unique.groupby(['User_id','Coupon_id'])
                                      ['Date_received','Date'].count()).reset_index()
coupon_user_redemption.columns = ['User_id','Coupon_id','User_Coupon_Released','User_Coupon_Redeemed']
coupon_user_redemption['User_Coupon_Ratio'] = coupon_user_redemption['User_Coupon_Redeemed']
/ coupon_user_redemption['User_Coupon_Released']
coupon_user_ratio = coupon_user_redemption[['User_id','Coupon_id','User_Coupon_Redeemed','User_Coupon_Ratio']]
coupon_user_ratio.head()

,User_id,Coupon_id,User_Coupon_Redeemed,User_Coupon_Ratio
0,4,2902.0,0,0
1,4,8735.0,0,0
2,35,1807.0,0,0
3,35,9776.0,0,0
4,35,11951.0,0,0


### Merchant-User Visits

In [24]:
#OFFLINE
visits_offline = pd.DataFrame(df_off_unique.groupby(['User_id','Merchant_id']).size()).reset_index()
visits_offline.columns = ['User_id','Merchant_id','Visits']
visits_offline.head()

,User_id,Merchant_id,Visits
0,4,1433,1
1,4,1469,1
2,35,3381,4
3,36,1041,1
4,36,5717,1


### Merchant-User Coupon Redemption Rate

In [25]:
merchant_user_redemption = pd.DataFrame(df_off_coupon.groupby(['User_id','Merchant_id'])['Date_received','Date'].count()).reset_index()
merchant_user_redemption.columns = ['User_id','Merchant_id','User_Merchant_Released','User_Merchant_Redeemed']
merchant_user_redemption['User_Merchant_Ratio'] = merchant_user_redemption['User_Merchant_Redeemed'] / merchant_user_redemption['User_Merchant_Released']
merchant_user_ratio = merchant_user_redemption[['User_id','Merchant_id','User_Merchant_Ratio']]
merchant_user_ratio

,User_id,Merchant_id,User_Merchant_Ratio
0,4,1433,0.0
1,4,1469,0.0
2,35,3381,0.0
3,36,1041,0.0
4,36,5717,0.0
...,...,...,...
856470,7360967,3103,0.0
856471,7360967,6485,0.0
856472,7361024,1433,0.0
856473,7361032,2146,0.0


### Merchant-User Last buying window

In [26]:
merchant_user_dates = df_off_unique[df_off_unique['Date'].isna()==False]
merchant_user_dates['First_day'] = pd.to_datetime('20160101',format='%Y%m%d')
merchant_user_dates['DayNum'] = merchant_user_dates['Date'] - merchant_user_dates['First_day'] 
merchant_user_dates['DayNum'] = merchant_user_dates['DayNum'].dt.days.astype('str')
merchant_user_dates['DayNum'] = pd.to_numeric(merchant_user_dates['DayNum'],errors="coerce") + 1
merchant_user_dates

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Count,First_day,DayNum
15,165,2934,NaN,NaN,0,NaT,2016-01-11,1,2016-01-01,11
16,165,2934,NaN,NaN,0,NaT,2016-01-25,1,2016-01-01,25
17,165,2934,NaN,NaN,0,NaT,2016-03-21,1,2016-01-01,81
18,165,2934,NaN,NaN,0,NaT,2016-03-28,1,2016-01-01,88
19,165,2934,NaN,NaN,0,NaT,2016-04-14,1,2016-01-01,105
...,...,...,...,...,...,...,...,...,...,...
1716977,7360941,6485,NaN,NaN,0,NaT,2016-03-25,1,2016-01-01,85
1716981,7360967,760,NaN,NaN,1,NaT,2016-06-25,1,2016-01-01,177
1716982,7360967,760,NaN,NaN,1,NaT,2016-06-26,1,2016-01-01,178
1716985,7360967,3103,NaN,NaN,2,NaT,2016-01-10,1,2016-01-01,10


In [27]:
merchant_user_days = pd.DataFrame(merchant_user_dates.groupby(['User_id','Merchant_id'])['DayNum'].apply(list).reset_index(name='Merchant_User_Visit'))


In [28]:
merchant_user_days['Merchant_User_Visit'] = merchant_user_days['Merchant_User_Visit'].apply(lambda x : sorted(set(x)))
merchant_user_days

,User_id,Merchant_id,Merchant_User_Visit
0,165,2934,"[11, 25, 81, 88, 105, 131, 154, 169]"
1,165,4195,"[97, 103, 111, 116, 139, 146]"
2,184,3381,[59]
3,209,3267,[157]
4,215,129,[63]
...,...,...,...
288543,7360928,3532,"[26, 96, 116, 132, 143, 148]"
288544,7360931,1469,[28]
288545,7360941,6485,"[85, 86, 87]"
288546,7360967,760,"[177, 178]"


### User Coupon Last Redemption Window

In [29]:
coupon_user_dates = df_off_coupon.copy()
coupon_user_dates['First_day'] = pd.to_datetime('20160101',format='%Y%m%d')
coupon_user_dates['DayNum'] = coupon_user_dates['Date_received'] - coupon_user_dates['First_day'] 
coupon_user_dates['DayNum'] = coupon_user_dates['DayNum'].dt.days.astype('str')
coupon_user_dates['DayNum'] = pd.to_numeric(coupon_user_dates['DayNum'],errors="coerce") + 1
coupon_user_dates.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Count,Discount,Original_price,Discounted_price,Rate,First_day,DayNum
0,4,1433,8735.0,"[30, 5, 0.833]",10,2016-02-14,NaT,1,30:5,30,5,0.833,2016-01-01,45
1,4,1469,2902.0,"[100, 5, 0.95]",10,2016-06-07,NaT,1,0.95,100,5,0.950,2016-01-01,159
2,35,3381,1807.0,"[300, 30, 0.9]",0,2016-01-30,NaT,1,300:30,300,30,0.900,2016-01-01,30
3,35,3381,9776.0,"[10, 5, 0.5]",0,2016-01-29,NaT,1,10:5,10,5,0.500,2016-01-01,29
4,35,3381,11951.0,"[200, 20, 0.9]",0,2016-01-29,NaT,1,200:20,200,20,0.900,2016-01-01,29


In [30]:
coupon_user_days = pd.DataFrame(coupon_user_dates.groupby(['User_id','Coupon_id'])['DayNum'].apply(list).reset_index(name='Coupon_User_Visit'))


In [31]:
coupon_user_days['Coupon_User_Visit'] = coupon_user_days['Coupon_User_Visit'].apply(lambda x : sorted(set(x)))
coupon_user_days.head()

,User_id,Coupon_id,Coupon_User_Visit
0,4,2902.0,[159]
1,4,8735.0,[45]
2,35,1807.0,[30]
3,35,9776.0,[29]
4,35,11951.0,"[29, 30]"


## Adding Features Train DataSet

In [32]:
#Adding user level features to training data
train_dataset = df_off_coupon.merge(users_level,how='left',on='User_id')
print(train_dataset.shape[0])

#Adding merchant level features to training data
train_dataset = pd.merge(train_dataset, merchants_level, how='left', on='Merchant_id')
print(train_dataset.shape[0])

#Adding coupon level features to training data
train_dataset = pd.merge(train_dataset, coupons_level, how='left', on='Coupon_id')
print(train_dataset.shape[0])

#Adding date received level features to training data
train_dataset = pd.merge(train_dataset, date_level, how='left', on='Date_received')
print(train_dataset.shape[0])

# Adding user_merchants visits
train_dataset = pd.merge(train_dataset,visits_offline)
print(train_dataset.shape[0])

# Adding user_merchants redemption rate
train_dataset = pd.merge(train_dataset,merchant_user_ratio)
print(train_dataset.shape[0])
train_dataset['User_Merchant_Ratio'].fillna(0,inplace=True)

# Adding user_coupon redemption rate
train_dataset = pd.merge(train_dataset,coupon_user_ratio)
print(train_dataset.shape[0])

# Adding user_discount redemption rate
train_dataset = pd.merge(train_dataset,user_discount_ratio, how='left')
print(train_dataset.shape[0])

# Adding user_merchants last buying window
train_dataset = pd.merge(train_dataset,merchant_user_days,how='left', on=['User_id','Merchant_id'])
print(train_dataset.shape[0])
for row in train_dataset.loc[train_dataset.Merchant_User_Visit.isnull(), 'Merchant_User_Visit'].index:
    train_dataset.at[row, 'Merchant_User_Visit'] = []
    
# Adding user_coupon last redeeming window
train_dataset = pd.merge(train_dataset,coupon_user_days,how='left', on=['User_id','Coupon_id'])
print(train_dataset.shape[0])
for row in train_dataset.loc[train_dataset.Coupon_User_Visit.isnull(), 'Coupon_User_Visit'].index:
    train_dataset.at[row, 'Coupon_User_Visit'] = []
    
    
train_dataset.head()

1015389
1015389
1015389
1015389
1015389
1015389
1015389
1015389
1015389
1015389


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Count,Discount,Original_price,...,Weekend,DayOfWeek,Visits,User_Merchant_Ratio,User_Coupon_Redeemed,User_Coupon_Ratio,Discounted_Redeemed,User_discount_Ratio,Merchant_User_Visit,Coupon_User_Visit
0,4,1433,8735.0,"[30, 5, 0.833]",10,2016-02-14,NaT,1,30:5,30,...,1,6,1,0.0,0,0,0,0.0,[],[45]
1,4,1469,2902.0,"[100, 5, 0.95]",10,2016-06-07,NaT,1,0.95,100,...,0,1,1,0.0,0,0,0,0.0,[],[159]
2,35,3381,1807.0,"[300, 30, 0.9]",0,2016-01-30,NaT,1,300:30,300,...,1,5,4,0.0,0,0,0,0.0,[],[30]
3,35,3381,9776.0,"[10, 5, 0.5]",0,2016-01-29,NaT,1,10:5,10,...,0,4,4,0.0,0,0,0,0.0,[],[29]
4,35,3381,11951.0,"[200, 20, 0.9]",0,2016-01-29,NaT,1,200:20,200,...,0,4,4,0.0,0,0,0,0.0,[],"[29, 30]"


In [33]:
from ast import literal_eval

## Adding last recent buying window of user

In [34]:
def lastVisit(days, x, check):
    if check:
        days = literal_eval(days)
    n = len(days)
    if n==0:
        return np.nan
    x = int(x)
    if x in days:
        try:
            i = days.index(x)
            if i == 0:
                return np.nan
            return days[i]-days[i-1] 
        except IndexError:
            return np.nan
    if x > days[n-1]:
        return x - days[n-1]
    elif x < days[0]:
        return np.nan
    else:
        for i in range(n):
            if (days[i]>x) & (i>=1):
                return x - days[i-1]
    return np.nan

In [35]:
train_dataset.loc[:,('DateTrack')] = train_dataset.loc[:,('Date_received')]
# train_dataset.DateTrack.fillna(train_dataset.Date_received, inplace=True)
train_dataset['First_day'] = pd.to_datetime('20160101',format='%Y%m%d')
train_dataset['DayNum'] = train_dataset['DateTrack'] - train_dataset['First_day'] 
train_dataset['DayNum'] = train_dataset['DayNum'].dt.days.astype('str')
train_dataset['DayNum'] = pd.to_numeric(train_dataset['DayNum'],errors="coerce") + 1


In [36]:
train_dataset['LastUserVisit'] = [lastVisit(x,y, True) for (x,y) in zip(train_dataset['DayList'],train_dataset['DayNum'])]

## Adding coupon last redemption window

In [37]:
train_dataset['LastRedemption'] = [lastVisit(x,y, True) for (x,y) in 
                                   zip(train_dataset['RedemptionList'],train_dataset['DayNum'])]

## Merchant and its last visiting window

In [38]:
train_dataset['LastMerchantVisit'] = [lastVisit(x,y, True) for (x,y) in 
                                      zip(train_dataset['VisitList'],train_dataset['DayNum'])]

## Merchant and its last redemption window

In [39]:
train_dataset['LastMerchantRedemption'] = [lastVisit(x,y, True) for (x,y) in 
                                           zip(train_dataset['MerchantRedemptionList'],train_dataset['DayNum'])]

## Merchant-User and its last buying window

In [40]:
train_dataset['LastMerchantUserVisit'] = [lastVisit(x,y, False) for (x,y) in 
                                          zip(train_dataset['Merchant_User_Visit'],train_dataset['DayNum'])]

## Coupon-User and its last redeeming window

In [41]:
train_dataset['LastCouponUserVisit']=[lastVisit(x,y, False) for (x,y) in 
                                          zip(train_dataset['Coupon_User_Visit'],train_dataset['DayNum'])]

In [42]:
train_dataset.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'Count', 'Discount', 'Original_price',
       'Discounted_price', 'Rate', 'Tag', 'User_Released', 'User_Redeemed',
       'User_Ratio', 'User_Buys', 'Purchaser', 'UserMerchantCount', 'DayList',
       'User_Redeemed_Buy', 'Merchant_Redeemed', 'Merchant_Ratio',
       'Merchant_AvgDistance', 'Merchant_Popular', 'Merchant_AvgRate',
       'AvgDailyUsers', 'VisitList', 'UniqueUsersCount', 'Merchant_Buys',
       'MerchantRedemptionList', 'Merchant_Redeemed_Buy', 'Coupon_Released',
       'Coupon_Redeemed', 'Coupon_Ratio', 'Duration', 'RedemptionList',
       'ReleasesCount', 'ImpDay', 'Weekend', 'DayOfWeek', 'Visits',
       'User_Merchant_Ratio', 'User_Coupon_Redeemed', 'User_Coupon_Ratio',
       'Discounted_Redeemed', 'User_discount_Ratio', 'Merchant_User_Visit',
       'Coupon_User_Visit', 'DateTrack', 'First_day', 'DayNum',
       'LastUserVisit', 'LastRedemption', 'LastMerchant

## Target Label 
    0: Coupon not Redeemed within 15 Days
    1: Coupon Redeemed within 15 Days

In [43]:
train_dataset['Date'].fillna(pd.to_datetime('20161201',format='%Y%m%d'),inplace=True)
train_dataset['RedemptionDuration'] = train_dataset['Date'] - train_dataset['Date_received']
train_dataset['RedemptionDuration'] = train_dataset['RedemptionDuration'].dt.days.astype('str')
train_dataset['RedemptionDuration'] = pd.to_numeric(train_dataset['RedemptionDuration'],errors="coerce")
train_dataset['Target'] = [1 if x<=15 else 0 for x in train_dataset['RedemptionDuration']]
train_dataset.shape

(1015389, 60)

In [44]:
print('Percentage of positive labels in training data: ')
print(str(round((train_dataset[train_dataset['Target']==1].shape[0]/train_dataset.shape[0])* 100, 2))+"%")
print(train_dataset[train_dataset['Target']==1].shape[0])

Percentage of positive labels in training data: 
6.12%
62153


In [45]:
train_dataset.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'Count', 'Discount', 'Original_price',
       'Discounted_price', 'Rate', 'Tag', 'User_Released', 'User_Redeemed',
       'User_Ratio', 'User_Buys', 'Purchaser', 'UserMerchantCount', 'DayList',
       'User_Redeemed_Buy', 'Merchant_Redeemed', 'Merchant_Ratio',
       'Merchant_AvgDistance', 'Merchant_Popular', 'Merchant_AvgRate',
       'AvgDailyUsers', 'VisitList', 'UniqueUsersCount', 'Merchant_Buys',
       'MerchantRedemptionList', 'Merchant_Redeemed_Buy', 'Coupon_Released',
       'Coupon_Redeemed', 'Coupon_Ratio', 'Duration', 'RedemptionList',
       'ReleasesCount', 'ImpDay', 'Weekend', 'DayOfWeek', 'Visits',
       'User_Merchant_Ratio', 'User_Coupon_Redeemed', 'User_Coupon_Ratio',
       'Discounted_Redeemed', 'User_discount_Ratio', 'Merchant_User_Visit',
       'Coupon_User_Visit', 'DateTrack', 'First_day', 'DayNum',
       'LastUserVisit', 'LastRedemption', 'LastMerchant

### Adding varaiables to track the first time users, merchants and coupons

In [46]:
train_dataset['FirstTimeUser'] = [1 if x==x else 0 for x in train_dataset['LastUserVisit']]
train_dataset['FirstTimeMerchant'] =[1 if x==x else 0 for x in train_dataset['LastMerchantVisit']]
train_dataset['FirstTimeCoupon'] = [1 if x==x else 0 for x in train_dataset['LastRedemption']]
train_dataset['FirstTimeMerchantUser'] = [1 if x==x else 0 for x in train_dataset['LastMerchantUserVisit']]
train_dataset['FirstTimeCouponUser'] =[1 if x==x else 0 for x in train_dataset['LastCouponUserVisit']]

In [47]:
train_dataset.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Count,Discount,Original_price,...,LastMerchantRedemption,LastMerchantUserVisit,LastCouponUserVisit,RedemptionDuration,Target,FirstTimeUser,FirstTimeMerchant,FirstTimeCoupon,FirstTimeMerchantUser,FirstTimeCouponUser
0,4,1433,8735.0,"[30, 5, 0.833]",10,2016-02-14,2016-12-01,1,30:5,30,...,1.0,NaN,NaN,291,0,0,1,1,0,0
1,4,1469,2902.0,"[100, 5, 0.95]",10,2016-06-07,2016-12-01,1,0.95,100,...,1.0,NaN,NaN,177,0,1,1,1,0,0
2,35,3381,1807.0,"[300, 30, 0.9]",0,2016-01-30,2016-12-01,1,300:30,300,...,1.0,NaN,NaN,306,0,0,1,1,0,0
3,35,3381,9776.0,"[10, 5, 0.5]",0,2016-01-29,2016-12-01,1,10:5,10,...,1.0,NaN,NaN,307,0,0,1,0,0,0
4,35,3381,11951.0,"[200, 20, 0.9]",0,2016-01-29,2016-12-01,1,200:20,200,...,1.0,NaN,NaN,307,0,0,1,1,0,0


In [48]:
train_dataset.to_csv('DataSets/DatasetsCreated/train_dataset.csv',index=False) 

## Adding Features Test DataSet

In [49]:
#Adding user level features to test data
test_dataset = df_test.merge(users_level,how='left',on='User_id')
print(test_dataset.shape[0])

#Adding merchant level features to training data
test_dataset = pd.merge(test_dataset, merchants_level, how='left', on='Merchant_id')
print(test_dataset.shape[0])

#Adding coupon level features to training data
test_dataset = pd.merge(test_dataset, coupons_level, how='left', on='Coupon_id')
print(test_dataset.shape[0])

#Adding date received level features to training data
test_dataset = pd.merge(test_dataset, date_level_test, how='left', on='Date_received')
print(test_dataset.shape[0])

# Adding user_merchants visits
test_dataset = pd.merge(test_dataset,visits_offline, how='left')
print(test_dataset.shape[0])

# Adding user_merchants redemption rate
test_dataset = pd.merge(test_dataset,merchant_user_ratio, how='left')
print(test_dataset.shape[0])
test_dataset['User_Merchant_Ratio'].fillna(0,inplace=True)

# Adding user_coupon redemption rate
test_dataset = pd.merge(test_dataset,coupon_user_ratio, how='left')
print(test_dataset.shape[0])


# Adding user_discount redemption rate
test_dataset = pd.merge(test_dataset,user_discount_ratio, how='left')
print(test_dataset.shape[0])


# Adding user_merchants last buying window
test_dataset = pd.merge(test_dataset,merchant_user_days,how='left', on=['User_id','Merchant_id'])
print(test_dataset.shape[0])

# Adding user_coupon last redeeming window
test_dataset = pd.merge(test_dataset,coupon_user_days,how='left', on=['User_id','Coupon_id'])
print(test_dataset.shape[0])

# FILLING NAN
for row in test_dataset.loc[test_dataset.Merchant_User_Visit.isnull(), 'Merchant_User_Visit'].index:
    test_dataset.at[row, 'Merchant_User_Visit'] = []

for row in test_dataset.loc[test_dataset.DayList.isnull(), 'DayList'].index:
    test_dataset.at[row, 'DayList'] = '[]'
    
for row in test_dataset.loc[test_dataset.VisitList.isnull(), 'VisitList'].index:
    test_dataset.at[row, 'VisitList'] = '[]'
    
for row in test_dataset.loc[test_dataset.RedemptionList.isnull(), 'RedemptionList'].index:
    test_dataset.at[row, 'RedemptionList'] = '[]'

for row in test_dataset.loc[test_dataset.MerchantRedemptionList.isnull(), 'MerchantRedemptionList'].index:
    test_dataset.at[row, 'MerchantRedemptionList'] = '[]'

for row in test_dataset.loc[test_dataset.Coupon_User_Visit.isnull(), 'Coupon_User_Visit'].index:
    test_dataset.at[row, 'Coupon_User_Visit'] = []
    

for i in ['User_Released','User_Redeemed', 'User_Ratio','User_Buys', 'Purchaser', 'UserMerchantCount',
          'Merchant_Redeemed','Merchant_Ratio','Merchant_AvgDistance', 'Merchant_Popular','Visits',
          'AvgDailyUsers', 'UniqueUsersCount', 'Coupon_Released', 'Coupon_Redeemed', 'Coupon_Ratio',
          'User_Merchant_Ratio','Duration']:
    test_dataset[i].fillna(0,inplace=True)
          
test_dataset.head()

113640
113640
113640
113640
113640
113640
113640
113640
113640
113640


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Discount,Original_price,Discounted_price,Rate,...,Weekend,DayOfWeek,Visits,User_Merchant_Ratio,User_Coupon_Redeemed,User_Coupon_Ratio,Discounted_Redeemed,User_discount_Ratio,Merchant_User_Visit,Coupon_User_Visit
0,4129537,450,9983,"[30, 5, 0.833]",1.0,2016-07-12,30:5,30,5,0.833,...,0,1,1.0,0.0,NaN,NaN,0.0,0.0,[],[]
1,6949378,1300,3429,"[30, 5, 0.833]",NaN,2016-07-06,30:5,30,5,0.833,...,0,2,0.0,0.0,NaN,NaN,1.0,1.0,[],[]
2,2166529,7113,6928,"[200, 20, 0.9]",5.0,2016-07-27,200:20,200,20,0.900,...,0,2,0.0,0.0,NaN,NaN,NaN,NaN,[],[]
3,2166529,7113,1808,"[100, 10, 0.9]",5.0,2016-07-27,100:10,100,10,0.900,...,0,2,0.0,0.0,NaN,NaN,NaN,NaN,[],[]
4,6172162,7605,6500,"[30, 1, 0.967]",2.0,2016-07-08,30:1,30,1,0.967,...,0,4,0.0,0.0,NaN,NaN,NaN,NaN,[],[]


In [50]:
test_dataset.loc[:,('DateTrack')] = test_dataset.loc[:,('Date_received')]
test_dataset['First_day'] = pd.to_datetime('20160101',format='%Y%m%d')
test_dataset['DayNum'] = test_dataset['DateTrack'] - test_dataset['First_day'] 
test_dataset['DayNum'] = test_dataset['DayNum'].dt.days.astype('str')
test_dataset['DayNum'] = pd.to_numeric(test_dataset['DayNum'],errors="coerce") + 1

In [51]:
test_dataset['LastUserVisit'] = [lastVisit(x,y, True) for (x,y) in zip(test_dataset['DayList'],test_dataset['DayNum'])]

## Adding coupon last redemption window (TEST Data)

In [52]:
test_dataset['LastRedemption'] = [lastVisit(x,y, True) for (x,y) in 
                                   zip(test_dataset['RedemptionList'],test_dataset['DayNum'])]

## Merchant and its last visiting window (TEST Data)

In [53]:
test_dataset['LastMerchantVisit'] = [lastVisit(x,y, True) for (x,y) in 
                                      zip(test_dataset['VisitList'],test_dataset['DayNum'])]

## Merchant and its last redemption window (TEST Data)

In [54]:
test_dataset['LastMerchantRedemption'] = [lastVisit(x,y, True) for (x,y) in 
                                           zip(test_dataset['MerchantRedemptionList'],test_dataset['DayNum'])]

## Merchant-User and its last buying window (TEST Data)

In [55]:
test_dataset['LastMerchantUserVisit'] = [lastVisit(x,y, False) for (x,y) in 
                                          zip(test_dataset['Merchant_User_Visit'],test_dataset['DayNum'])]

## Coupon-User and its last redeeming window

In [56]:
test_dataset['LastCouponUserVisit']=[lastVisit(x,y, False) for (x,y) in 
                                          zip(test_dataset['Coupon_User_Visit'],test_dataset['DayNum'])]

### Adding varaiables to track the first time users, merchants and coupons

In [57]:
test_dataset['FirstTimeUser'] = [1 if x==x else 0 for x in test_dataset['LastUserVisit']]
test_dataset['FirstTimeMerchant'] =[1 if x==x else 0 for x in test_dataset['LastMerchantVisit']]
test_dataset['FirstTimeCoupon'] = [1 if x==x else 0 for x in test_dataset['LastRedemption']]
test_dataset['FirstTimeMerchantUser'] = [1 if x==x else 0 for x in test_dataset['LastMerchantUserVisit']]
test_dataset['FirstTimeCouponUser'] =[1 if x==x else 0 for x in test_dataset['LastCouponUserVisit']]

In [58]:
test_dataset.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Discount', 'Original_price', 'Discounted_price',
       'Rate', 'Tag', 'User_Released', 'User_Redeemed', 'User_Ratio',
       'User_Buys', 'Purchaser', 'UserMerchantCount', 'DayList',
       'User_Redeemed_Buy', 'Merchant_Redeemed', 'Merchant_Ratio',
       'Merchant_AvgDistance', 'Merchant_Popular', 'Merchant_AvgRate',
       'AvgDailyUsers', 'VisitList', 'UniqueUsersCount', 'Merchant_Buys',
       'MerchantRedemptionList', 'Merchant_Redeemed_Buy', 'Coupon_Released',
       'Coupon_Redeemed', 'Coupon_Ratio', 'Duration', 'RedemptionList',
       'ReleasesCount', 'ImpDay', 'Weekend', 'DayOfWeek', 'Visits',
       'User_Merchant_Ratio', 'User_Coupon_Redeemed', 'User_Coupon_Ratio',
       'Discounted_Redeemed', 'User_discount_Ratio', 'Merchant_User_Visit',
       'Coupon_User_Visit', 'DateTrack', 'First_day', 'DayNum',
       'LastUserVisit', 'LastRedemption', 'LastMerchantVisit',
       'L

In [59]:
test_dataset.shape, train_dataset.shape

((113640, 61), (1015389, 65))

In [60]:
set(train_dataset.columns) - set(test_dataset.columns)

{'Count', 'Date', 'RedemptionDuration', 'Target'}

In [61]:
test_dataset.to_csv('DataSets/DatasetsCreated/test_dataset.csv',index=False) 

# Execution Time of this notebook

In [62]:
later = time.time()
difference = later - now
print('Time taken for the execution of this notebook: '+str(round(difference/60,2))+' mins')

Time taken for the execution of this notebook: 16.69 mins
